#### Importamos Librerias

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

#### Levantamos dataset

In [ ]:
df = pd.read_csv("../../data/preprocessed/base.csv")
df["periodo_dt"] = pd.to_datetime(df["periodo"].astype(str), format="%Y%m")
df.shape

# Productos
#### Periodo de productos

Vemos cuantos productos perduraron a lo largo del tiempo

In [ ]:
# Suponemos que ya tenés cargado tu DataFrame df con columnas: periodo (AAAAMM), customer_id, product_id, tn
# Convertimos periodo a datetime


# Paso 2: Determinar periodo de vida útil de cada producto
vida_producto = df.groupby("product_id")["periodo_dt"].agg(["min", "max"]).reset_index()


# Función para calcular meses exactos
def meses_entre_fechas(f1, f2):
    return (f2.year - f1.year) * 12 + (f2.month - f1.month)

# Calcular duración en meses exactos
vida_producto['duracion_meses'] = vida_producto.apply(
    lambda x: meses_entre_fechas(x['min'], x['max']), 
    axis=1
)


# Calcular duración máxima en meses (asegurando cubrir todo el rango)
max_meses = vida_producto['duracion_meses'].max()
bins = int(max_meses) + 1  # +1 para incluir el último mes

# Crear el gráfico con bins exactos por mes
plt.figure(figsize=(14, 7))
ax = sns.histplot(
    data=vida_producto,
    x='duracion_meses',
    bins=bins,  # ¡Ajuste clave aquí!
    discrete=True,  # Para que cada barra sea un mes entero
    color='#4e79a7',
    edgecolor='white',
    alpha=0.8
)

# Añadir etiquetas a las barras
for p in ax.patches:
    height = p.get_height()
    if height > 0:
        ax.annotate(
            f'{int(height)}',
            (p.get_x() + p.get_width() / 2., height),
            ha='center', va='center',
            xytext=(0, 5),
            textcoords='offset points',
            fontsize=9
        )

# Personalización avanzada
plt.title('Distribución Exacta de Vida Útil por Mes', pad=20, fontsize=14)
plt.xlabel('Duración en Meses', fontsize=12)
plt.ylabel('Cantidad de Productos', fontsize=12)
plt.xticks(np.arange(0, max_meses + 1, step=3))  # Muestra cada 3 meses
plt.grid(axis='y', linestyle='--', alpha=0.3)

# Ajustar ejes
ax.set_xlim(-0.5, max_meses + 0.5)  # Margen para visualización

plt.tight_layout()
plt.show()

product_id    927
dtype: int64

In [ ]:
vida_producto[vida_producto['max']>='2019-10-01'].shape